In [16]:
import pandas as pd
import requests
from tqdm.notebook import tqdm

import dandelion

In [2]:
!wget https://files.deeppavlov.ai/deeppavlov_data/entity_linking/test_dataset.pickle

--2022-06-28 12:58:57--  https://files.deeppavlov.ai/deeppavlov_data/entity_linking/test_dataset.pickle
Resolving files.deeppavlov.ai (files.deeppavlov.ai)... 178.63.27.41
Connecting to files.deeppavlov.ai (files.deeppavlov.ai)|178.63.27.41|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17668715 (17M) [application/octet-stream]
Saving to: ‘test_dataset.pickle’

test_dataset.pickle 100%[===================>]  16.85M  9.37MB/s    in 1.8s    

2022-06-28 12:58:59 (9.37 MB/s) - ‘test_dataset.pickle’ saved [17668715/17668715]



## Download pickled dataset and prepare it for scoring

In [3]:
import pickle

with open("test_dataset.pickle", "rb") as dataset_f:
    dataset = pickle.load(dataset_f)

full_dataset = []
for kind, value_list in dataset.items():
    data_rows = [{"kind": kind, "full_text": value} for value in value_list]
    full_dataset += data_rows

dataset_df = pd.DataFrame(full_dataset)
slice_size = 3500
max_text_len = dataset_df["full_text"].str.len().max()
max_slices = max_text_len // slice_size

for s_idx in range(max_slices):
    start = s_idx * slice_size
    stop = start + slice_size
    dataset_df[f"text_{s_idx}"] = dataset_df["full_text"].str.slice(start, stop)

dataset_df = dataset_df.melt(id_vars=["kind", "full_text"], value_name="text")
dataset_df.replace("", float("NaN"), inplace=True)
dataset_df.dropna(subset=["text"], inplace=True)
dataset_df.reset_index(inplace=True)
dataset_df[["kind", "text"]].to_csv("test_dataset.csv", sep="|", index=False)

In [4]:
!rm test_dataset.pickle

In [5]:
dataset_df = pd.read_csv("test_dataset.csv", sep="|")
dataset_df

,kind,text
0,news_texts,Chicago church embraces ‘The Gospel According ...
1,news_texts,Opinion | Endorsements for Montgomery County C...
2,news_texts,"Arch Manning, nephew of Peyton and Eli Manning..."
3,news_texts,"NBA draft winners and losers: Banchero, Smith ..."
4,news_texts,"Gaetz, Brooks, Biggs, Gohmert, Perry accused o..."
...,...,...
6101,blog_texts,shing minerality on the palate and blossom sce...
6102,blog_texts,he QR code located on the site or on any 2020 ...
6103,blog_texts,baker’s chocolate with a hint of acidity.Fresh...
6104,blog_texts,egan with eco-friendly packaging — sustainable...


In [6]:
def run_dandelion(df, fresh_start=True):
    results = []

    with requests.Session() as sess:
        for idx, row in tqdm(df.iterrows(), total=len(df)):
            if not pd.isna(row.get("dandelion_results", float("NaN"))):
                continue

            data = error = exception = float("NaN")
            try:
                result = dandelion.extract_dandelion(sess, row["text"])
                if result.error:
                    error = result.error
                    if error.get("code") in ["error.authenticationError", "error.forbiddenError"]:
                        print("Exceeded API token limit, come back tomorrow!")
                        break
                else:
                    data = result.data
            except Exception as e:
                exception = f"{type(e)}: {e}"
            results.append({"index": idx, "dandelion_results": data, "dandelion_error": error, "dandelion_script_exception": exception})

    if results:
        results_df = pd.DataFrame(results)
        results_df.set_index("index", inplace=True)
        if fresh_start:
            results_df = pd.merge(df, results_df, left_index=True, right_index=True, how="left")
        else:
            df.update(results_df, overwrite=False)
            results_df = df

        results_df.to_csv("test_dataset_dandelion.csv", sep="|", index=False)
    else:
        results_df = df

    return results_df


## Fresh run
Execute this cell only if you don't have a test_dataset_dandelion.csv yet

In [9]:
# Fresh run
dandelion_results_df = run_dandelion(dataset_df, fresh_start=True)
dandelion_results_df

  0%|          | 0/6106 [00:00<?, ?it/s]/home/mtalimanchuk/.virtualenvs/entity_extraction_svc/lib/python3.7/site-packages/cryptography/hazmat/backends/openssl/x509.py:17: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  utils.DeprecatedIn35,
 18%|█▊        | 1088/6106 [04:26<15:29,  5.40it/s]  

Exceeded API token limit, come back tomorrow!


,kind,text,dandelion_results,dandelion_error,script_exception
0,news_texts,Chicago church embraces ‘The Gospel According ...,"{'time': 100, 'annotations': [{'start': 0, 'en...",NaN,NaN
1,news_texts,Opinion | Endorsements for Montgomery County C...,"{'time': 92, 'annotations': [{'start': 27, 'en...",NaN,NaN
2,news_texts,"Arch Manning, nephew of Peyton and Eli Manning...","{'time': 138, 'annotations': [{'start': 0, 'en...",NaN,NaN
3,news_texts,"NBA draft winners and losers: Banchero, Smith ...","{'time': 119, 'annotations': [{'start': 30, 'e...",NaN,NaN
4,news_texts,"Gaetz, Brooks, Biggs, Gohmert, Perry accused o...","{'time': 157, 'annotations': [{'start': 0, 'en...",NaN,NaN
...,...,...,...,...,...
6101,blog_texts,shing minerality on the palate and blossom sce...,NaN,NaN,NaN
6102,blog_texts,he QR code located on the site or on any 2020 ...,NaN,NaN,NaN
6103,blog_texts,baker’s chocolate with a hint of acidity.Fresh...,NaN,NaN,NaN
6104,blog_texts,egan with eco-friendly packaging — sustainable...,NaN,NaN,NaN


## Consecutive runs
Run this if you already have a test_dataset_dandelion.csv from previous runs

In [7]:
# Run again using dandelion_results_df to update missing results
dandelion_results_df = pd.read_csv("test_dataset_dandelion.csv", sep="|")
dandelion_results_df = run_dandelion(dandelion_results_df, fresh_start=False)
dandelion_results_df

  0%|          | 0/6106 [00:00<?, ?it/s]/home/mtalimanchuk/.virtualenvs/entity_extraction_svc/lib/python3.7/site-packages/cryptography/hazmat/backends/openssl/x509.py:17: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  utils.DeprecatedIn35,
 38%|███▊      | 2319/6106 [03:35<10:30,  6.01it/s]  

Exceeded API token limit, come back tomorrow!


,kind,text,dandelion_results,dandelion_error,script_exception
0,news_texts,Chicago church embraces ‘The Gospel According ...,"{'time': 100, 'annotations': [{'start': 0, 'en...",NaN,NaN
1,news_texts,Opinion | Endorsements for Montgomery County C...,"{'time': 92, 'annotations': [{'start': 27, 'en...",NaN,NaN
2,news_texts,"Arch Manning, nephew of Peyton and Eli Manning...","{'time': 138, 'annotations': [{'start': 0, 'en...",NaN,NaN
3,news_texts,"NBA draft winners and losers: Banchero, Smith ...","{'time': 119, 'annotations': [{'start': 30, 'e...",NaN,NaN
4,news_texts,"Gaetz, Brooks, Biggs, Gohmert, Perry accused o...","{'time': 157, 'annotations': [{'start': 0, 'en...",NaN,NaN
...,...,...,...,...,...
6101,blog_texts,shing minerality on the palate and blossom sce...,NaN,NaN,NaN
6102,blog_texts,he QR code located on the site or on any 2020 ...,NaN,NaN,NaN
6103,blog_texts,baker’s chocolate with a hint of acidity.Fresh...,NaN,NaN,NaN
6104,blog_texts,egan with eco-friendly packaging — sustainable...,NaN,NaN,NaN


In [33]:
def run_deeppavlov(df, url, slice_start=None, slice_stop=None):
    df_slice = df[slice_start:slice_stop]
    results = []

    with requests.Session() as sess:
        for idx, row in tqdm(df_slice.iterrows(), total=len(df_slice)):
            data = error = exception = float("NaN")
            try:
                response = sess.post(url, json={"text": row["text"]})
                result = response.json()
                if result.get("detail"):
                    error = result
                else:
                    data = result
            except Exception as e:
                exception = f"{type(e)}: {e}"
            results.append({"index": idx, "dp_results": data, "dp_error": error, "dp_script_exception": exception})

    # if results:
    results_df = pd.DataFrame(results)
    results_df.set_index("index", inplace=True)
    results_df = pd.merge(df, results_df, left_index=True, right_index=True, how="left")

    results_df.to_csv("test_dataset_dp.csv", sep="|", index=False)
    # else:
    #     results_df = df

    return results_df

In [34]:
dp_results_df = run_deeppavlov(dataset_df, "http://10.11.1.6:9999/", 0, 2000)
dp_results_df

,kind,text,dp_results,dp_error,script_exception
0,news_texts,Chicago church embraces ‘The Gospel According ...,NaN,NaN,<class 'json.decoder.JSONDecodeError'>: Expect...
1,news_texts,Opinion | Endorsements for Montgomery County C...,"{'annotations': [{'start': 27, 'end': 52, 'spo...",NaN,NaN
2,news_texts,"Arch Manning, nephew of Peyton and Eli Manning...","{'annotations': [{'start': 0, 'end': 12, 'spot...",NaN,NaN
3,news_texts,"NBA draft winners and losers: Banchero, Smith ...","{'annotations': [{'start': 0, 'end': 3, 'spot'...",NaN,NaN
4,news_texts,"Gaetz, Brooks, Biggs, Gohmert, Perry accused o...","{'annotations': [{'start': 56, 'end': 61, 'spo...",NaN,NaN
...,...,...,...,...,...
6101,blog_texts,shing minerality on the palate and blossom sce...,NaN,NaN,NaN
6102,blog_texts,he QR code located on the site or on any 2020 ...,NaN,NaN,NaN
6103,blog_texts,baker’s chocolate with a hint of acidity.Fresh...,NaN,NaN,NaN
6104,blog_texts,egan with eco-friendly packaging — sustainable...,NaN,NaN,NaN


In [38]:
comparison_df = pd.merge(dandelion_results_df, dp_results_df, left_index=True, right_index=True, how="left")
comparison_df

,kind_x,text_x,dandelion_results,dandelion_error,script_exception_x,kind_y,text_y,dp_results,dp_error,script_exception_y
0,news_texts,Chicago church embraces ‘The Gospel According ...,"{'time': 100, 'annotations': [{'start': 0, 'en...",NaN,NaN,news_texts,Chicago church embraces ‘The Gospel According ...,NaN,NaN,<class 'json.decoder.JSONDecodeError'>: Expect...
1,news_texts,Opinion | Endorsements for Montgomery County C...,"{'time': 92, 'annotations': [{'start': 27, 'en...",NaN,NaN,news_texts,Opinion | Endorsements for Montgomery County C...,"{'annotations': [{'start': 27, 'end': 52, 'spo...",NaN,NaN
2,news_texts,"Arch Manning, nephew of Peyton and Eli Manning...","{'time': 138, 'annotations': [{'start': 0, 'en...",NaN,NaN,news_texts,"Arch Manning, nephew of Peyton and Eli Manning...","{'annotations': [{'start': 0, 'end': 12, 'spot...",NaN,NaN
3,news_texts,"NBA draft winners and losers: Banchero, Smith ...","{'time': 119, 'annotations': [{'start': 30, 'e...",NaN,NaN,news_texts,"NBA draft winners and losers: Banchero, Smith ...","{'annotations': [{'start': 0, 'end': 3, 'spot'...",NaN,NaN
4,news_texts,"Gaetz, Brooks, Biggs, Gohmert, Perry accused o...","{'time': 157, 'annotations': [{'start': 0, 'en...",NaN,NaN,news_texts,"Gaetz, Brooks, Biggs, Gohmert, Perry accused o...","{'annotations': [{'start': 56, 'end': 61, 'spo...",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
6101,blog_texts,shing minerality on the palate and blossom sce...,NaN,NaN,NaN,blog_texts,shing minerality on the palate and blossom sce...,NaN,NaN,NaN
6102,blog_texts,he QR code located on the site or on any 2020 ...,NaN,NaN,NaN,blog_texts,he QR code located on the site or on any 2020 ...,NaN,NaN,NaN
6103,blog_texts,baker’s chocolate with a hint of acidity.Fresh...,NaN,NaN,NaN,blog_texts,baker’s chocolate with a hint of acidity.Fresh...,NaN,NaN,NaN
6104,blog_texts,egan with eco-friendly packaging — sustainable...,NaN,NaN,NaN,blog_texts,egan with eco-friendly packaging — sustainable...,NaN,NaN,NaN


In [74]:
import ast

comparison_df = comparison_df.dropna(subset=["dandelion_results", "dp_results"], how="any")
for idx, row in comparison_df[:10].iterrows():
    dandelion_substrings = {(ann["start"], ann["end"]): ann["spot"] for ann in ast.literal_eval(row["dandelion_results"])["annotations"]}
    dp_substrings = {(ann["start"], ann["end"]): ann["spot"] for ann in row["dp_results"]["annotations"]}

    negative_diff = set(dandelion_substrings) - set(dp_substrings)
    positive_diff = set(dp_substrings) - set(dandelion_substrings)
    print(f"- {';'.join(dandelion_substrings[nd] for nd in negative_diff)}")
    print(f"+ {';'.join(dp_substrings[pd] for pd in positive_diff)}")
    print()

- rent control;PostDemocracy;pedestrian safety;learning;Germantown;affordable housing;actions;businesses;local;residential property;crime;business;businesses;property management;Chamber of Commerce;lawyer;state flag;covid-19;public transportation;Gaithersburg;seats;Montgomery County;term-limited;Op-ed;funds;The Washington Post
+ Washington Post Accessibility;Goldberg;Marilyn Balcombe;Gabe Albornoz;election;primary;the County Council;Montgomery;Goldberg;Germantown Chamber;Scott Goldberg;Andrew Friedson;ballot;primary;Evan Glass;Montgomery;Commerce

- pick;Arch Manning;quarterback;Peyton;Arch Manning;basketball;footwork;Isidore Newman School;quarterback;Arch Manning;Maalik;Twitter;touchdown;starter;quarterbacks;college football;Warner;Manning;freshman;2009-10 season;grandson;Peyton;Eli Manning;passes;The Washington Post;varsity;pocket presence;Matt Leinart;PostDemocracy;quarterback;Manning family
+ college;com;Greenies;football;MaxPreps;season;season;Isidore Newman;Ranked;Eli;Matt Bonest